# Overview

The task at hand is classification of wine quality

We will use 3 different approaches:

- A standard neural network (feed fordward nn)
- A bayesian neural network that will take into account epistemic (model) uncertainty on the predicted labels
- A probabilistic neural network that will take into account both aleatoric (data) and epistemic (model) uncertainty on the predicted labels

## Workflow

1. [Data Inspection](#inspection) 
    - Loading
    - Inspection
    - Preprocessing
2. [Modeling](#model-definition)
    - Standard Neural Network
    - Bayesian Neural Network
    - Probabilistic Neural Network
3. [Prediction](#prediction)

In [1]:
# Software install (as required)
#!pip install -r ../requirements.txt

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns

2023-05-24 15:25:45.819146: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 15:25:47.818817: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 15:25:47.819960: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 15:25:51.825633: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading and Inspection <a name="inspection"></a>

In [17]:
def load_data(dataset_name="wine_quality",buffer_size=4998,split="train",batch_size=256,train_size=3000):
    ds , ds_info = tfds.load(name=dataset_name, as_supervised=True, split=split ,with_info=True)
    
    (ds.map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=buffer_size)
        .cache()
    )
    # Train : we shuffle with a buffer the same size as the dataset.
    ds_train = (
        ds
        .take(train_size)
        .shuffle(buffer_size=train_size)
        .batch(batch_size)
    )
    # Test : no shuffle
    ds_test = (
        ds
        .skip(train_size)
        .batch(batch_size)
    )

    return ds_train, ds_test, ds_info

In [18]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
ds_train,ds_test ,ds_info = load_data(
    dataset_name="wine_quality",
    buffer_size=dataset_size, # We prefetch with a buffer the same size as the dataset because th dataset is very small and fits into memory.
    batch_size=batch_size,
    train_size=train_size
    ) 
    

In [19]:
# Basic Info
feature_names=list(ds_info.features['features'].keys())
print("Total examples: %d" %(len(ds_train)+len(ds_test)))
print("Train set size: %d" %len(ds_train)) 
print("Test set size : %d" %len(ds_test))   
print("Feature names : %s" %feature_names)
print("")

Total examples: 20
Train set size: 17
Test set size : 3
Feature names : ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']



In [24]:
# show a few examples from the train dataset
tfds.as_dataframe(ds_train.unbatch().take(10), ds_info)

2023-05-24 15:48:44.657983: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-05-24 15:48:44.658565: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]


,features/alcohol,features/chlorides,features/citric acid,features/density,features/fixed acidity,features/free sulfur dioxide,features/pH,features/residual sugar,features/sulphates,features/total sulfur dioxide,features/volatile acidity,quality
0,8.899999618530273,0.04699999839067459,0.20000000298023224,0.9976000189781189,5.800000190734863,26.0,3.0899999141693115,16.049999237060547,0.46000000834465027,166.0,0.33000001311302185,5
1,11.0,0.050999999046325684,0.3499999940395355,0.991599977016449,6.199999809265137,24.0,3.369999885559082,0.699999988079071,0.4300000071525574,111.0,0.23000000417232513,3
2,12.399999618530273,0.01600000075995922,0.41999998688697815,0.9900699853897095,6.099999904632568,31.0,3.1500000953674316,5.0,0.3100000023841858,113.0,0.3799999952316284,7
3,9.800000190734863,0.04600000008940697,0.30000001192092896,0.9941999912261963,8.699999809265137,29.0,3.2200000286102295,1.600000023841858,0.3799999952316284,130.0,0.15000000596046448,6
4,9.699999809265137,0.06400000303983688,0.28999999165534973,0.9973700046539307,7.099999904632568,56.0,3.1600000858306885,15.5,0.4099999964237213,115.5,0.12999999523162842,7
5,9.399999618530273,0.04800000041723251,0.23999999463558197,0.9957000017166138,7.0,31.0,3.2300000190734863,6.199999809265137,0.6200000047683716,228.0,0.3199999928474426,6
6,11.75,0.03099999949336052,0.25,0.9907199740409851,5.300000190734863,45.0,3.309999942779541,3.9000000953674316,0.5799999833106995,130.0,0.4000000059604645,7
7,9.800000190734863,0.05400000140070915,0.3700000047683716,0.99795001745224,8.0,23.0,3.319999933242798,9.600000381469727,0.4699999988079071,159.0,0.23000000417232513,4
8,8.899999618530273,0.06599999964237213,0.41999998688697815,0.9979000091552734,7.400000095367432,48.0,2.890000104904175,14.0,0.41999998688697815,198.0,0.23999999463558197,6
9,11.800000190734863,0.032999999821186066,0.30000001192092896,0.9905999898910522,6.300000190734863,16.0,3.2799999713897705,1.7999999523162842,0.4000000059604645,91.0,0.23000000417232513,6


In [74]:
# Class balance check : is the dataset imbalanced?
#fig, ax = plt.subplots(1, 1, figsize=(10,6))
#labels, counts = np.unique(np.fromiter(ds_train.unbatch().map(lambda x, y: y), np.int32),  return_counts=True)
#ax.set_xlabel('Counts')
#ax.set_title("Counts by type");
#sns.barplot(x=counts, y=[class_names[l] for l in labels], label="Total")
#ax.grid(True,ls='--')
#sns.despine(left=True, bottom=True)

In [75]:
def prepare_for_training(ds, cache=True, batch_size=1, shuffle_buffer_size=1000):
  ds = ds.map(lambda x, y: (x, tf.cast(y, tf.float32)))
  ds = ds.prefetch(buffer_size=4898)
  ds = ds.cache()
  # shuffle the dataset
  ds = ds.shuffle(buffer_size=shuffle_buffer_size)
  # split to batches
  ds = ds.batch(batch_size)
  # `prefetch` lets the dataset fetch batches in the background while the model is training.
  return ds

In [76]:
batch_size = 1
# preprocess training & validation sets
ds_train = prepare_for_training(ds_train, batch_size=batch_size,shuffle_buffer_size=len(ds_train))

In [29]:
# Function to create model inputs
def create_model_inputs():
    inputs = {}
    for name in feature_names:
        inputs[name] = layers.Input(
            name=name.replace(" ","_"), shape=(1,), dtype=tf.float32
        )
    return inputs

# Create Standard Neural Network
def base_neural_network(hidden_units=None):
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


# Function to train and evaluate a model (experiment run)
def run_experiment(model, loss, train_dataset, test_dataset, num_epochs, learning_rate,save=True,model_filename=""):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Model training started ...")
    model.fit(
        train_dataset, 
        epochs=num_epochs, 
        validation_data=test_dataset)
    
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

    # save model as required
    if save:
        print('saving model : %s' %model_filename)
        model.save(model_filename)


In [30]:
arch_type = 'nn'
model_filename = "wine_quality_classification_"+arch_type
model_path = os.path.join("../models", model_filename + ".h5")
if not os.path.exists("../models"):
    os.makedirs(model_path)

### Neural Network Training <a name="model training"></a>

In [31]:
hidden_units = [8, 8]
learning_rate = 0.001
num_epochs = 100
nn_model = base_neural_network(hidden_units=hidden_units)
run_experiment(
    model=nn_model, 
    loss=keras.losses.MeanSquaredError(), 
    train_dataset=ds_train, 
    test_dataset=ds_test,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
    model_filename=model_filename)


Model training started ...
Epoch 1/100
17/17 [==============================] - 1s 9ms/step - loss: 38.3675 - root_mean_squared_error: 6.1942 - val_loss: 33.9572 - val_root_mean_squared_error: 5.8273
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 35.8448 - root_mean_squared_error: 5.9871 - val_loss: 32.4479 - val_root_mean_squared_error: 5.6963
Epoch 3/100
17/17 [==============================] - 0s 4ms/step - loss: 33.7326 - root_mean_squared_error: 5.8080 - val_loss: 30.6022 - val_root_mean_squared_error: 5.5319
Epoch 4/100
17/17 [==============================] - 0s 4ms/step - loss: 31.7324 - root_mean_squared_error: 5.6332 - val_loss: 28.6952 - val_root_mean_squared_error: 5.3568
Epoch 5/100
17/17 [==============================] - 0s 4ms/step - loss: 29.8150 - root_mean_squared_error: 5.4603 - val_loss: 26.8533 - val_root_mean_squared_error: 5.1820
Epoch 6/100
17/17 [==============================] - 0s 4ms/step - loss: 27.9766 - root_mean_squared_error: 

INFO:tensorflow:Assets written to: wine_quality_classification_nn/assets


INFO:tensorflow:Assets written to: wine_quality_classification_nn/assets


We take a sample from the test set use the model to obtain predictions for them. Note that since the baseline model is deterministic, we get a single a point estimate prediction for each test example, with no information about the uncertainty of the model nor the prediction.

In [32]:
sample = 10
examples, targets = list(ds_test.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = base_neural_network(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")


2023-05-24 15:59:25.811594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-05-24 15:59:25.812405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]


ValueError: invalid literal for int() with base 10: 'alcohol'